Koristimo biblioteku networkx za rad sa grafovima. Radimo za sad za graf koji je dat kao primer u pdf-u. Prilagoditi kasnije za testiranje sa drugačijim grafovima.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
G.add_edge('v1', 'v2')
G.add_edge('v2', 'v3')
G.add_edge('v2', 'v4')
G.add_edge('v4', 'v5')

pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos,node_color='white')
nx.draw_networkx_labels(G,pos,font_color='black')
nx.draw_networkx_edges(G,pos,edgelist=G.edges,color='red')
plt.axis('off')
plt.show()

/home/nevena/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


<Figure size 640x480 with 1 Axes>

Funkcija labeliranja $f:{v_1,v_2,...,v_n} \rightarrow {1,2,...,n}$
Neka je $f(v_1) = 3, f(v_2) = 1, f(v_3) = 2, f(v_4) = 5, f(v_5) = 4 $ 

In [5]:
f = [3,1,2,5,4] # v1 -> 3, v2 -> 1, ...
nodes = [u for u in G.nodes] # v1, v2, v3, v4, v5
print('Given labeling: ',f)
print('Nodes: ',nodes)
print('Neighbors of a node v2: ',[u for u in G.neighbors('v2')])


Given labeling:  [3, 1, 2, 5, 4]
Nodes:  ['v1', 'v2', 'v3', 'v4', 'v5']
Neighbors of a node v2:  ['v1', 'v3', 'v4']


Definišemo funkciju $label$ koja za dati graf $G$, čvor $node$ i funkciju labeliranja $f$ vraća oznaku (labelu) koju tom čvoru dodeljuje funkcija $f$

In [6]:
def label(G, node, f):
    nodes = [v for v in G.nodes]
    index_of_node = nodes.index(node)
    return f[index_of_node]

print('Label of a node v2 according to f: ', label(G,'v2',f))


Label of a node v2 according to f:  1


Definišemo funkciju $bandwidth\_of\_a\_node$ koja za dati graf $G$, čvor $node$ i funkciju labeliranja $f$ izračunava $bandwidth$ tog čvora 

In [7]:
def bandwidth_of_a_node(G, node, f):
    neighbors = [u for u in G.neighbors(node)]
    label_node = label(G,node,f)
    maximum = -1
    for u in neighbors:
        label_u = label(G,u,f)
        diff = abs(label_u - label_node)
        if diff > maximum:
            maximum = diff
    return maximum

print('Bandwidth of node v2 under labeling f is',bandwidth_of_a_node(G,'v2',f))
print('Bandwidth of node v3 under labeling f is',bandwidth_of_a_node(G,'v3',f))

Bandwidth of node v2 under labeling f is 4
Bandwidth of node v3 under labeling f is 1


Definišemo funkciju $bandwidth$ koja za dati graf $G$ i funkciju labeliranja $f$ izračunava $bandwidth$ grafa 

In [8]:
# racunamo bandwidth grafa G za funkciju f
def bandwidth(G, f):
    nodes = [v for v in G.nodes]
    bandwidths = [bandwidth_of_a_node(G,node,f) for node in nodes]
    return max(bandwidths)

print('Bandwidth of the graph G under labeling f is',bandwidth(G,f))
print('Bandwidth of the graph G under identic labeling is',bandwidth(G,[1,2,3,4,5]))

Bandwidth of the graph G under labeling f is 4
Bandwidth of the graph G under identic labeling is 2


Definišemo dve pomoćne funkcije. $labeling\_from\_nodes$ je pomoćna funkcija koja nam od liste čvorova $v_j$ daje listu indeksa $j$ (tj. labeliranje tih čvorova).
$nodes\_from\_labeling$ je pomoćna funkcija koja nam od liste indeksa $j$ daje listu čvorova $v_j$ (tj. čvorove na osnovu tog labeliranja).

In [9]:
def labeling_from_nodes(nodes):
    n = len(nodes)
    return [int(nodes[i].strip('v')) for i in range(n)]

def nodes_from_labeling(labeling):
    n = len(labeling)
    return ['v'+str(labeling[i]) for i in range(n)]

Definišemo funkciju initialSolution koja za dati graf $G$ generiše početno rešenje - početno labeliranje $f$ čvorova grafa $G$, koristeći BFS sa nasumičnim izborom čvora od kojeg se započinje pretraga.

In [10]:
import random
    
def initialSolution(G):
    nodes = [u for u in G.nodes]
    start = random.choice(nodes)
    edges_iterator = nx.bfs_edges(G,start)
    edges = [edge for edge in edges_iterator]
    solution = [start] + [edge[1] for edge in edges]
    return labeling_from_nodes(solution)

initSol = initialSolution(G)
print('Is our initial solution better than previous labeling f?')
if bandwidth(G,initSol) < bandwidth(G,f):
    print('YES!')
else:
    print('NO!')

Is our initial solution better than previous labeling f?
YES!


Definišemo funkciju $distance(f,f_p)$ koja računa rastojanje između rešenja $f$ i $f_p$ na osnovu formule (4) iz pdf-a. Potrebna da bi definisali pojam okoline nekog rešenja.

In [11]:
def distance(f,f_p):
    n = len(f)
    dist = 0
    for i in range(n):
        if f[i] != f_p[i]:
            dist += 1
    dist -= 1
    return dist
        
f = [3,2,1,5,4]
f_p = [4,1,3,2,5]
print('Distance between given solutions is',distance(f,f_p))

Distance between given solutions is 4


Definišemo funkcije $max\_labeled\_neighbor$ i $min\_labeled\_neighbor$ koje za dati graf $G$, čvor $node$ i labeliranje $f$ računaju, redom, najveću i najmanju labelu među čvorovima susednim čvoru $node$

In [12]:
def max_labeled_neighbor(G, node, f):
    neighbors = [u for u in G.neighbors(node)]
    max_label = float('-inf')
    for u in neighbors:
        label_u = label(G,u,f)
        if label_u > max_label:
            max_label = label_u
    return max_label

def min_labeled_neighbor(G, node, f):
    neighbors = [u for u in G.neighbors(node)]
    min_label = float('inf')
    for u in neighbors:
        label_u = label(G,u,f)
        if label_u < min_label:
            min_label = label_u
    return min_label

In [13]:
f = [3,1,2,5,4]
print(max_labeled_neighbor(G,'v2',f))
print(min_labeled_neighbor(G,'v2',f))

5
2


Definišemo funkciju $set\_K$ koja bira podskup čvorova $K$ na osnovu $Initialization$ koraka u $Algorithm 1$ pdf-a. 
### ZA OVO NISAM SIGURNA!!!!

In [14]:
def set_K(G, f, k):
    nodes = [u for u in G.nodes]
    B_p = bandwidth(G,f)
    K = []
    while True:
        for node in nodes:
            bandwidth_node = bandwidth_of_a_node(G, node, f)
            if bandwidth_node >= B_p:
                K.append(node)
                
        if len(K) >= k:
            break
        else:
            K = []
            B_p -= 1
                
    return K
                
set_K(G, f, 4)

['v1', 'v2', 'v3', 'v4', 'v5']

 Definišemo funkciju $critical\_node$ koja za dati graf $G$, čvor $node$ i labeliranje $f$ nalazi čvor $v$ takav da je $ |f(node)-f(v)| = bandwidth\_of\_a\_node(G, node, f) $

In [15]:
def critical_node(G, node, f):
    
    nodes = [u for u in G.nodes]
    
    for v in nodes:
        if abs(label(G, node, f) - label(G, v, f)) == bandwidth_of_a_node(G, node, f):
            return v

Definišemo pomoćnu funkciju $swap\_labels$ koja ažurira labeliranje $f$ tako što datim čvorovima $u$ i $v$ zameni prethodne oznake.

In [19]:
def swap_labels(G, u, v, f):
    
    label_u = label(G, u, f)
    label_v = label(G, v, f)
    
    index_u = f.index(label_u)
    index_v = f.index(label_v)
    
    f[index_u] = label_v
    f[index_v] = label_u
    
    return f


In [20]:
def shaking(G, f, k):
    
    K = set_K(G, f, k)
    
    edges = [edge for edge in G.edges]
    nodes = [node for node in G.nodes]
    
    for i in range(1,k):
        u = random.choice(K)
        v = critical_node(G, u, f)
        if (u,v) in edges:
            f_min_u = min_labeled_neighbor(G, u, f)
            f_max_u = max_labeled_neighbor(G, u, f)
            label_v = label(G, v, f)
            
            min_value = float('inf')
            min_w = None
            
            for node in nodes:
                label_node = label(G, node, f)
                if label_node <= f_max_u and label_node >= f_min_u:
                    current_value = max(max_labeled_neighbor(G, w, f) - label_v,label_v-min_labeled_neighbor(G, w, f))
                    
                    if current_value < min_value:
                        min_value = current_value
                        min_w = node
                        
            f = swap_labels(f,u,v)
                        
                    
                
    